In [1]:


import os
#donnes
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.
import pandas as pd
import numpy as np
#random forest pour regressor
from sklearn.ensemble import RandomForestRegressor
#Pour le split feature et target
from sklearn.model_selection import train_test_split
#Pour regression au lieu de accuracy c'est mean_square_error
from sklearn.metrics import mean_squared_error
#Ne pas afficher le warning lors du fit par exemple
#Import pour la cross_validation
from sklearn.model_selection  import cross_val_score
#import random forest pour regression
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

['train.csv', 'sampleSubmission.csv', 'test.csv']


In [2]:
# Fichier de train
X_train = pd.read_csv("../input/train.csv")
#Fichier de test
X_test = pd.read_csv("../input/test.csv")

In [3]:
submission = pd.read_csv("../input/sampleSubmission.csv")
submission["count"] = 195

###FAIRE UNE SUBMISSION

In [4]:
#Convertir notre fichier en csv
submission.to_csv('submission.csv', index=False)

In [5]:
X_test.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed'],
      dtype='object')

In [6]:
## Definition foncyion features et target

def split_dataset(df, features, target='count'):
    X = df[features]
    y = df[target]
    return X, y


In [7]:
## Fonction split date
def date_split(df_train, df_test, date='datetime'):
    ##Traitement_df_train
    cols=df_train[date]
    date_cols=pd.to_datetime(cols)
    df_train['year'] = date_cols.dt.year
    df_train['month'] = date_cols.dt.month
    df_train['day'] = date_cols.dt.day
    df_train['hour'] = date_cols.dt.hour
    df_train['minute'] = date_cols.dt.minute
    df_train['second'] = date_cols.dt.second
    df_train = df_train.drop(['datetime'], axis=1)
    ##Traitement_df_test
    cols2=df_test[date]
    date_cols2=pd.to_datetime(cols2)
    df_test['year'] = date_cols2.dt.year
    df_test['month'] = date_cols2.dt.month
    df_test['day'] = date_cols2.dt.day
    df_test['hour'] = date_cols2.dt.hour
    df_test['minute'] = date_cols2.dt.minute
    df_test['second'] = date_cols2.dt.second
    df_test = df_test.drop(['datetime'], axis=1)
    return df_train, df_test
     
    
    

In [8]:
#Definition de X_train et X_test avec les memes columns
X_train, X_test = date_split(X_train, X_test)

In [9]:
#### definissons nos features numbers ( features qui sont que des nombres)
##ATTENTION: Avoir les mm colonnes avec le dataset de test
# Fonction de recuperation du meme type de colonne
def Get_cols(df, features_test=X_test.columns):
    ############################POUR X_TRAIN########################
    #Renvoyer features de X_test à  X_train
    X_train_features = df[features_test]
    return  X_train_features



In [10]:
#Appel de la fonction pour avoir le meme nombre de columns:
X_trainGet_cols = Get_cols(X_test)

numbers = X_trainGet_cols.select_dtypes(np.number)

numbers.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,hour,minute,second
0,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,0,0,0
1,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,1,0,0
2,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,2,0,0
3,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,0,0
4,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,4,0,0


In [11]:
X_train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0,0,0
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1,0,0
2,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2,0,0
3,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3,0,0
4,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4,0,0


In [12]:
##Definition features and target

X_train_features, y_train_target = split_dataset(X_train, features=numbers.columns)

X_train_features, y_train_target

(       season  holiday  workingday  weather   ...    day  hour  minute  second
 0           1        0           0        1   ...      1     0       0       0
 1           1        0           0        1   ...      1     1       0       0
 2           1        0           0        1   ...      1     2       0       0
 3           1        0           0        1   ...      1     3       0       0
 4           1        0           0        1   ...      1     4       0       0
 5           1        0           0        2   ...      1     5       0       0
 6           1        0           0        1   ...      1     6       0       0
 7           1        0           0        1   ...      1     7       0       0
 8           1        0           0        1   ...      1     8       0       0
 9           1        0           0        1   ...      1     9       0       0
 10          1        0           0        1   ...      1    10       0       0
 11          1        0           0     

In [15]:
##################Cross Validation
## random forest regressor
#Import Random Forest pour regressor
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
##Cross Validation
score = -cross_val_score(rf, X_train_features, y_train_target, cv=5, scoring='neg_mean_squared_error')

In [16]:
score.mean()

6146.539811967758

In [17]:
###############  FIT entrainer tout le set d'entrainement
rf.fit(X_train_features, y_train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
## Predict sur le train
y_train_pred = rf.predict(X_train_features)

ATTENTIONNNNNNNNNNNNNN: DANS KAGGLE LES COLUMNS DANS TEST PEUVENT ETRE SUPERIEUR OU INFERIEUR AU COLUMNS DANS LE DATASET DE TRAIN

In [19]:
###### Predict sur le test
## Voir si on a le meme nombre de columns dans test et dans train
y_test_pred = rf.predict(X_test)


In [23]:
mean_train = mean_squared_error( y_train_target, y_train_pred)
#mean_test = mean_squared_error(y_test, y_test_pred)
mean_train
#, mean_test

356.2379432298365

In [24]:
submission = pd.read_csv("../input/sampleSubmission.csv")
submission["count"] = y_test_pred
#Convertir notre fichier en csv
submission.to_csv('submission.csv', index=False)

In [22]:
#!rm submission.csv

In [25]:
submission.head(3)

,datetime,count
0,2011-01-20 00:00:00,13.7
1,2011-01-20 01:00:00,6.1
2,2011-01-20 02:00:00,4.9
